In [3]:
from langchain_openai import ChatOpenAI

In [4]:
import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"]= userdata.get('OPENAI_API_KEY')
os.environ['HF_HOME']= "/content/drive/MyDrive/models"

In [18]:
model1= ChatOpenAI()

In [8]:
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint

In [9]:
llm = HuggingFaceEndpoint(
    repo_id="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    task="text-generation"
)

In [10]:
model= ChatHuggingFace(llm=llm)

In [12]:
from langchain_core.prompts import PromptTemplate

In [13]:
template1= PromptTemplate(
    template= "Tell me details about {topic}",
    input_variables=["topic"]
)

In [14]:
template2= PromptTemplate(
    template= "Give me 5 line summary on below text.\n {text}",
    input_variables=["text"]
)

In [15]:
prompt1= template1.invoke({"topic":"Pydantic"})

In [17]:
from langchain_huggingface import ChatHuggingFace, HuggingFacePipeline
llm = HuggingFacePipeline.from_model_id(
    model_id="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    task="text-generation",
    model_kwargs={"temperature": 0, "max_length": 64}
)

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Device set to use cpu


In [19]:
model_hf= ChatHuggingFace(llm=llm)

In [20]:
result= model_hf.invoke(prompt1)

In [21]:
prompt2= template2.invoke({"text":result.content})

In [22]:
res= model_hf.invoke(prompt2)

In [23]:
print(res.content)

<|user|>
Give me 5 line summary on below text.
 <|user|>
Tell me details about Pydantic</s>
<|assistant|>
Pydantic is a Python library that helps developers create and validate models for data in a unified and flexible way. It is designed to work with Python's standard library and has a simple and intuitive API. Pydantic is a tool for building data models that provide a clear and consistent way to represent data in Python.

Pydantic is built on top of the Django Framework and is designed to work with Django models. Pydantic models can be used to define the schema of complex data models. The model class can be defined in a JSON or YAML file, and Pydantic can automatically generate the necessary Python classes for the data.

Pydantic also provides support for defining schemas for different data formats, such as JSON, XML, and CSV. Pydantic's JSON schema support allows for easy validation of data against a schema and provides a clear and consistent representation of data in JSON.

Pydanti

In [24]:
from langchain_core.output_parsers import StrOutputParser
parser= StrOutputParser()

In [28]:
chain = template1 | model_hf | parser | template2 | model_hf | parser

In [29]:
chain.invoke({"topic":"about you?"})

'<|user|>\nGive me 5 line summary on below text.\n <|user|>\nTell me details about about you?</s>\n<|assistant|>\nSure, here are some details about me:\n\n- My name is [Your Name] and I am [Your Age] years old.\n- I was born on [Birthday] in [City].\n- I am a [Job Title] at [Organization Name].\n- I studied [Degree] and [Years of Education] at [University Name].\n- I have [Number of Hobbies] hobbies and enjoy spending my free time playing [Hobby 1], [Hobby 2], and [Hobby 3].\n- My favorite color is [Favorite Color].\n- I love [Adjective] and [Adjective] together.\n- I am [Gender] and [Religion].\n- I am a [Relationship Status] and [Occupation/Job].\n- I have a [Number of Likes] on [Platform Name].\n- I am [Marital Status] and [Children].\n- My favorite food is [Favorite Food].\n- I am [Height] and [Weight].\n- I have a [Number of Pets] of [Number of Species].\n- I am a [Living Arrangement] and [Spending].\n\nI am a [Gender] and [Religion].\nMy favorite hobbies are [Hobbies], and I enjo

## JSON OutputParser

In [32]:
from langchain_core.output_parsers import JsonOutputParser

In [33]:
parser_json= JsonOutputParser()

In [34]:
template3= PromptTemplate(
    template= "Give me name,age and city of below fictional person.\n {format_instruction}",
    input_variables=[],
    partial_variables={"format_instruction":parser_json.get_format_instructions()}
)

In [35]:
prompt3= template3.format()
prompt3

'Give me name,age and city of below fictional person.\n Return a JSON object.'

In [36]:
result= model1.invoke(prompt3)

In [38]:
final_res= parser.parse(result.content)
final_res

'{\n  "name": "Sophia",\n  "age": 28,\n  "city": "Boston"\n}'

In [39]:
chain2= template3 | model_hf | parser_json

In [41]:
res_hf= chain2.invoke({})

In [42]:
res_hf

{'name': 'Jane Doe', 'age': 30, 'city': 'New York'}

## Pydantic OutputParser

In [43]:
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

In [47]:
class Car(BaseModel):
    name: str = Field(description="name of the car")
    color: str = Field(description="color of the car")
    price: float = Field(description="price of the car")
    brand: str = Field(description="brand of the car")

In [48]:
parser= PydanticOutputParser(pydantic_object=Car)

In [49]:
template= PromptTemplate(
    template= "Give me name, colour, price, brand of best selling car of {year}\n {format_instruction}",
    input_variables=["year"],
    partial_variables={"format_instruction":parser.get_format_instructions()}
)

In [57]:
chain= template | model1 | parser
final_res= chain.invoke({"year":"2024"})

In [58]:
final_res.dict()

{'name': 'Tesla Model 3', 'color': 'Black', 'price': 45000.0, 'brand': 'Tesla'}

In [ ]:
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage

In [ ]:
messages=[
    SystemMessage(content="You are helpful assistant"),
    HumanMessage(content="Tell me about Langchain")
    ]

In [ ]:
res= model.invoke(messages)

In [ ]:
messages.append(AIMessage(content=res.content))

In [ ]:
messages

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage

In [ ]:
chain_json = prompt3 | model_hf | parser_json
result_json = chain_json.invoke({})
display(result_json)